MCMC chain analysis
===========

In [6]:
%matplotlib inline
import numpy as np

The first thing we need to do is open the chain file.

In [9]:
raw = np.loadtxt('chain.dat');
shape = raw.size
print(shape)
    

40000


We know that the model has two parameters, so we need to reshape the chain, and find out how many steps were taken

In [12]:
save_dim = 2
num_steps = shape/(save_dim)
print(num_steps)

20000.0


Now we need to reshape the chain into something that we can use

In [19]:
chain = raw.reshape(num_steps,save_dim)
print(chain.shape)

(20000, 2)


/Users/davidparkinson/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


Now we can compute some summary statistics. Maybe we can look at the mean of the parameters